# Data Preprocessing

In [21]:
import sympy as sp
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import pad_sequences
from keras.utils import to_categorical
from sympy.printing.str import StrPrinter
import nltk
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout


# Define a symbol for the independent variable
x = sp.symbols('x')

# Define a list of functions to generate Taylor expansions for
functions = [sp.sin(x), sp.cos(x), sp.exp(x), sp.ln(1+x), sp.atan(x),sp.cot(x),sp.sec(x),sp.csc(x),sp.asin(x),sp.acot(x),sp.sinh(x),sp.cosh(x),sp.tanh(x)]
functions_str = ['sin(x)','cos(x)','exp(x)','ln(1+x)','atan(x)','cot(x)','sec(x)','csc(x)','asin(x)','acos(x)','acot(x)','sinh(x)','cosh(x)','tanh(x)'] #for tokenizing as StrPrinter() does not work directly on these functions

print(functions)

# Given order of expansion = 4
order = 4

# Dataset of functions and their Taylor expansions
data = []
for f in functions:
    taylor_series = f.series(x, 0, order+1).removeO()
    data.append((f, taylor_series))

# Print out the dataset
for f, t in data:
    print(f"Function: {f}")
    print(f"Taylor Series: {t}\n")

[sin(x), cos(x), exp(x), log(x + 1), atan(x), cot(x), sec(x), csc(x), asin(x), acot(x), sinh(x), cosh(x), tanh(x)]
Function: sin(x)
Taylor Series: -x**3/6 + x

Function: cos(x)
Taylor Series: x**4/24 - x**2/2 + 1

Function: exp(x)
Taylor Series: x**4/24 + x**3/6 + x**2/2 + x + 1

Function: log(x + 1)
Taylor Series: -x**4/4 + x**3/3 - x**2/2 + x

Function: atan(x)
Taylor Series: -x**3/3 + x

Function: cot(x)
Taylor Series: -x**3/45 - x/3 + 1/x

Function: sec(x)
Taylor Series: 5*x**4/24 + x**2/2 + 1

Function: csc(x)
Taylor Series: 7*x**3/360 + x/6 + 1/x

Function: asin(x)
Taylor Series: x**3/6 + x

Function: acot(x)
Taylor Series: x**3/3 - x + pi/2

Function: sinh(x)
Taylor Series: x**3/6 + x

Function: cosh(x)
Taylor Series: x**4/24 + x**2/2 + 1

Function: tanh(x)
Taylor Series: -x**3/3 + x



We have used these functions for Taylor expansion dataset

# Tokenization

In [22]:
# Generate a dataset of functions and their Taylor expansions
dataset = []
for f,g in zip(functions,functions_str):
    # Generate the Taylor series expansion
    taylor_series = f.series(x, 0, order+1).removeO()

    # Convert the expression to a string with desired format and tokenize it
    expr_str = StrPrinter().doprint(taylor_series).replace(' ','')

    expr_fn = StrPrinter().doprint(g)

    tokenized_expr = [c for c in expr_str]
    tokenized_fn = [fn for fn in expr_fn]

    tokenized_final = []
    for i in tokenized_fn:
      tokenized_final.append(i)
    for j in tokenized_expr:
      tokenized_final.append(j)

    dataset.append(tokenized_final)
dataset

[['s', 'i', 'n', '(', 'x', ')', '-', 'x', '*', '*', '3', '/', '6', '+', 'x'],
 ['c',
  'o',
  's',
  '(',
  'x',
  ')',
  'x',
  '*',
  '*',
  '4',
  '/',
  '2',
  '4',
  '-',
  'x',
  '*',
  '*',
  '2',
  '/',
  '2',
  '+',
  '1'],
 ['e',
  'x',
  'p',
  '(',
  'x',
  ')',
  'x',
  '*',
  '*',
  '4',
  '/',
  '2',
  '4',
  '+',
  'x',
  '*',
  '*',
  '3',
  '/',
  '6',
  '+',
  'x',
  '*',
  '*',
  '2',
  '/',
  '2',
  '+',
  'x',
  '+',
  '1'],
 ['l',
  'n',
  '(',
  '1',
  '+',
  'x',
  ')',
  '-',
  'x',
  '*',
  '*',
  '4',
  '/',
  '4',
  '+',
  'x',
  '*',
  '*',
  '3',
  '/',
  '3',
  '-',
  'x',
  '*',
  '*',
  '2',
  '/',
  '2',
  '+',
  'x'],
 ['a',
  't',
  'a',
  'n',
  '(',
  'x',
  ')',
  '-',
  'x',
  '*',
  '*',
  '3',
  '/',
  '3',
  '+',
  'x'],
 ['c',
  'o',
  't',
  '(',
  'x',
  ')',
  '-',
  'x',
  '*',
  '*',
  '3',
  '/',
  '4',
  '5',
  '-',
  'x',
  '/',
  '3',
  '+',
  '1',
  '/',
  'x'],
 ['s',
  'e',
  'c',
  '(',
  'x',
  ')',
  '5',
  '*',
  'x',
  '*',


# Padding

In [23]:
# Pad and convert the dataset to numpy arrays
max_len = max(len(string) for string in dataset)
dataset = pad_sequences(dataset, maxlen=max_len,dtype = 'str', padding='post', truncating='post',value = 0)
dataset = np.array(dataset)
print(dataset)
vocab_size = len(dataset[0]) + 1

[['s' 'i' 'n' '(' 'x' ')' '-' 'x' '*' '*' '3' '/' '6' '+' 'x' '0' '0' '0'
  '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0']
 ['c' 'o' 's' '(' 'x' ')' 'x' '*' '*' '4' '/' '2' '4' '-' 'x' '*' '*' '2'
  '/' '2' '+' '1' '0' '0' '0' '0' '0' '0' '0' '0' '0']
 ['e' 'x' 'p' '(' 'x' ')' 'x' '*' '*' '4' '/' '2' '4' '+' 'x' '*' '*' '3'
  '/' '6' '+' 'x' '*' '*' '2' '/' '2' '+' 'x' '+' '1']
 ['l' 'n' '(' '1' '+' 'x' ')' '-' 'x' '*' '*' '4' '/' '4' '+' 'x' '*' '*'
  '3' '/' '3' '-' 'x' '*' '*' '2' '/' '2' '+' 'x' '0']
 ['a' 't' 'a' 'n' '(' 'x' ')' '-' 'x' '*' '*' '3' '/' '3' '+' 'x' '0' '0'
  '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0']
 ['c' 'o' 't' '(' 'x' ')' '-' 'x' '*' '*' '3' '/' '4' '5' '-' 'x' '/' '3'
  '+' '1' '/' 'x' '0' '0' '0' '0' '0' '0' '0' '0' '0']
 ['s' 'e' 'c' '(' 'x' ')' '5' '*' 'x' '*' '*' '4' '/' '2' '4' '+' 'x' '*'
  '*' '2' '/' '2' '+' '1' '0' '0' '0' '0' '0' '0' '0']
 ['c' 's' 'c' '(' 'x' ')' '7' '*' 'x' '*' '*' '3' '/' '3' '6' '0' '+' 'x'
  '/' '6' '+' '1' '/' '

# One-Hot Encoding

In [24]:
for i in range(len(dataset)):
  dataset[i] = np.array(dataset[i])
  label_encoder = LabelEncoder()
  dataset[i] = label_encoder.fit_transform(dataset[i])

print(dataset)

[['1' '9' '1' '0' '1' '1' '4' '1' '2' '2' '7' '5' '8' '3' '1' '6' '6' '6'
  '6' '6' '6' '6' '6' '6' '6' '6' '6' '6' '6' '6' '6']
 ['1' '1' '1' '0' '1' '1' '1' '2' '2' '9' '5' '8' '9' '4' '1' '2' '2' '8'
  '5' '8' '3' '7' '6' '6' '6' '6' '6' '6' '6' '6' '6']
 ['1' '1' '1' '0' '1' '1' '1' '2' '2' '8' '4' '6' '8' '3' '1' '2' '2' '7'
  '4' '9' '3' '1' '2' '2' '6' '4' '6' '3' '1' '3' '5']
 ['1' '1' '0' '7' '3' '1' '1' '4' '1' '2' '2' '1' '5' '1' '3' '1' '2' '2'
  '9' '5' '9' '4' '1' '2' '2' '8' '5' '8' '3' '1' '6']
 ['8' '1' '8' '9' '0' '1' '1' '4' '1' '2' '2' '7' '5' '7' '3' '1' '6' '6'
  '6' '6' '6' '6' '6' '6' '6' '6' '6' '6' '6' '6' '6']
 ['1' '1' '1' '0' '1' '1' '4' '1' '2' '2' '8' '5' '9' '1' '4' '1' '5' '8'
  '3' '7' '5' '1' '6' '6' '6' '6' '6' '6' '6' '6' '6']
 ['1' '1' '1' '0' '1' '1' '9' '2' '1' '2' '2' '8' '4' '7' '8' '3' '1' '2'
  '2' '7' '4' '7' '3' '6' '5' '5' '5' '5' '5' '5' '5']
 ['1' '1' '1' '0' '1' '1' '9' '2' '1' '2' '2' '7' '4' '7' '8' '5' '3' '1'
  '4' '8' '3' '6' '4' '

In [25]:
dataset = to_categorical(dataset, num_classes=vocab_size)
print(dataset)

[[[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  ...
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


# LSTM Model

Each token is represented as a one-hot encoded vector, so the output of the model is a probability distribution over the vocabulary, indicating the likelihood of each token being the next one in the sequence. Therefore, the task can be formulated as a multi-class classification problem, where the model aims to predict the class (i.e., the next token) of the input sequence and thus,categorical_cross entropy as loss fn. can be used. 2 dropout layers are considered as to prevent overfitting which is extremely likely as the dataset,i.e,number of functions are extremely limited for training

In [26]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(units=64, input_shape=(max_len, vocab_size), return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(Dense(units=vocab_size, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy')

# Train the model
model.fit(dataset, dataset, epochs=100, validation_split=0.2, batch_size=32)

Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 3.4823 - val_loss: 3.4603
Epoch 2/100
1/1 [==============================] - 0s 46ms/step - loss: 3.4627 - val_loss: 3.4416
Epoch 3/100
1/1 [==============================] - 0s 43ms/step - loss: 3.4448 - val_loss: 3.4225
Epoch 4/100
1/1 [==============================] - 0s 40ms/step - loss: 3.4253 - val_loss: 3.4024
Epoch 5/100
1/1 [==============================] - 0s 39ms/step - loss: 3.4052 - val_loss: 3.3803
Epoch 6/100
1/1 [==============================] - 0s 41ms/step - loss: 3.3829 - val_loss: 3.3553
Epoch 7/100
1/1 [==============================] - 0s 39ms/step - loss: 3.3613 - val_loss: 3.3262
Epoch 8/100
1/1 [==============================] - 0s 46ms/step - loss: 3.3344 - val_loss: 3.2915
Epoch 9/100
1/1 [==============================] - 0s 43ms/step - loss: 3.2978 - val_loss: 3.2491
Epoch 10/100
1/1 [==============================] - 0s 41ms/step - loss: 3.2560 - val_loss: 3.1964
Epoch 11/100
1/1 [===

In [27]:
model.predict(dataset)

1/1 [==============================] - 1s 570ms/step


array([[[4.32062075e-02, 5.38484603e-02, 3.85917164e-02, ...,
         2.93814056e-02, 2.96735000e-02, 2.66744737e-02],
        [5.88935986e-02, 1.10211223e-01, 5.25391772e-02, ...,
         2.50928402e-02, 2.67015994e-02, 1.99875906e-02],
        [7.79164508e-02, 2.82721967e-01, 7.47500658e-02, ...,
         1.48279173e-02, 1.73124485e-02, 9.62352101e-03],
        ...,
        [4.56533395e-04, 6.15830952e-03, 1.30564021e-03, ...,
         5.11325052e-05, 1.22141646e-04, 1.07306427e-04],
        [4.54129244e-04, 6.12458307e-03, 1.29796600e-03, ...,
         5.09966630e-05, 1.21934761e-04, 1.06892010e-04],
        [4.52423701e-04, 6.10142155e-03, 1.29245152e-03, ...,
         5.08907797e-05, 1.21787940e-04, 1.06600004e-04]],

       [[4.32062075e-02, 5.38484603e-02, 3.85917164e-02, ...,
         2.93814056e-02, 2.96735000e-02, 2.66744737e-02],
        [6.21036999e-02, 1.25720814e-01, 5.38586862e-02, ...,
         2.44825184e-02, 2.62484681e-02, 1.86902545e-02],
        [8.06065947e-02, 

### These values above are the corresponding normalized probability distributions

## Observations and Inferences:-
I have tried to prevent overfitting using 2 dropout layers (Overfitting may be reduced further by k fold cross val. if it still exists).
Although we have obtained the corresponding probability distributions by training the model, an intuitive feel of how well has the model performed is hard to visualize as the results are one hot encoded which are expected as input for the LSTM model.This one hot encoding is essential due to categorical data.
We are also taking the functions themselves as input along with their corresponding taylor expansions to map the essential relationships between the function and the series.
I could increase the number of functions in the dataset input to make the model more efficient,although the exact number of functions to be taken was not mentioned in the problem statement.